In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer



from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from numpy import arange

In [3]:
train=pd.read_csv(r'/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test=pd.read_csv(r'/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
train.dropna(axis=0, subset=['SalePrice'], inplace=True)
y=train.SalePrice
train.drop(['SalePrice'], axis=1, inplace=True)

X_train_full,X_valid_full, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=0)

num_cols=[col for col in train.columns if train[col].dtype!=object]
cat_cols=[col for col in train.columns if train[col].dtype==object and train[col].nunique()<10]
mycols=num_cols+cat_cols

In [5]:


X_train=X_train_full[mycols].copy()
X_valid=X_valid_full[mycols].copy()
X_test=test[mycols].copy()

In [6]:
num_transform=SimpleImputer(strategy='mean')
cat_transform=Pipeline(steps=[('imp', SimpleImputer(strategy='most_frequent')),
                              ('oe',OneHotEncoder(handle_unknown='ignore'))
                               ])
                              
ct=ColumnTransformer(transformers=[('num',num_transform, num_cols),
                                   ('cat',cat_transform, cat_cols)
    
])
                              
model= XGBRegressor(n_jobs=5)

mypipe= Pipeline(steps=[
    ('preprocessor', ct),
    ('xgb',model)
])




In [7]:
# fit_params={'model__early_stopping_rounds': 30, 
#             'model__verbose': False,
#             'model__eval_set': [(X_valid, y_valid)]}

# scores = -1 * cross_val_score(mypipe, X_train, y_train,
#                               cv=5, 
#                               scoring='neg_mean_absolute_error', 
#                               )

# print("MAE scores:\n", scores.mean())

In [8]:
param_grid = {
    "xgb__n_estimators": [100,500,1000],
    "xgb__learning_rate": [0.01, 0.05, 0.1]
    
}

# params = {"mypipe__xgb__eval_set": [(X_valid, y_valid)], 
#               "mypipe__xgb__early_stopping_rounds": 10, 
#               "mypipe__xgb__verbose": False}

searchCV = GridSearchCV(mypipe, cv=5, param_grid=param_grid, scoring='neg_mean_squared_error')
searchCV.fit(X_train, y_train) 
              

best_n_estimators = searchCV.best_params_['xgb__n_estimators']
best_learning_rate = searchCV.best_params_['xgb__learning_rate']

In [9]:
print(f'Best learning rate is {best_learning_rate}. Best number of estimators is {best_n_estimators}.')
print(f'best score is {np.sqrt(-searchCV.best_score_)}')

Best learning rate is 0.1. Best number of estimators is 500.
best score is 27989.274549332848


In [10]:
mypipe= Pipeline(steps=[
    ('preprocessor', ct),
    ('xgb',XGBRegressor(n_estimators=500, learning_rate=0.1, n_jobs=5))
])

# mypipe.fit(X_train,y_train, eval_set=[(X_valid,y_valid)], verbose=False)

scores = np.sqrt(-1 * cross_val_score(mypipe, X_train, y_train,
                              cv=5,
                              scoring='neg_mean_squared_error'))

print("MSE scores:\n", scores.mean())

MSE scores:
 27109.467972448656


In [11]:
final_X=pd.concat([X_train,X_valid], axis=0)
final_y=pd.concat([y_train,y_valid], axis=0)

scores = np.sqrt(-1 * cross_val_score(mypipe, final_X, final_y,
                              cv=5,
                              scoring='neg_mean_squared_error'))
print("MSE scores:\n", scores.mean())

MSE scores:
 27936.45915891581


In [12]:
mypipe.fit(final_X,final_y)

submissions5=mypipe.predict(X_test)
submissions_5=pd.DataFrame({'Id': test['Id'],
                          'SalePrice': submissions5
                           })


In [13]:
submissions_5.head()

,Id,SalePrice
0,1461,125206.992188
1,1462,154928.937500
2,1463,177393.796875
3,1464,192756.609375
4,1465,188503.656250


In [14]:
submissions_5.to_csv('xgb_pipeline_2.csv', index=False)